In [10]:
import sys
from pathlib import Path
import pandas as pd

# Ensure root is on path
sys.path.append(str(Path().resolve().parent))

from backtesting.data.loader import load_csv_data
from backtesting.strategy.momentum import MomentumStrategy
from backtesting.engine.backtester import BacktestRunner

# Load data
prices = load_csv_data("../backtesting/data/spy.csv")

# Run strategy
strategy = MomentumStrategy(lookback=20)
bt = BacktestRunner(prices, strategy, slippage_bps=5)
results = bt.run()

# View results
print(results)

{'Annualized Return': np.float64(19.709525982216114), 'Sharpe Ratio': np.float64(5.2092221510854655), 'Max Drawdown': np.float64(-0.009457370883112057)}
